In [2]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

import polars as pl

pd.set_option('display.max_columns', None)

In [15]:
esquema_hospitales = pd.read_excel(
    "../data/external/Esquema_Registro-2023.xls", sheet_name=4, header=7
).dropna(how="all", axis=1)[["Código nuevo Establecimiento", "Nombre Oficial"]]

In [16]:
df = (
    (
        pl.scan_csv("../data/processed/df_procesada.csv", separator=";")
        .filter((pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN"))
        .group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL"]))
        .agg(
            pl.col("DIAGNOSTICO1").count().alias("n_egresos"),
            pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
            pl.col("IR_29301_PESO").mean().alias("promedio_peso_grd"),
        )
    )
    .collect()
    .to_pandas()
    .sort_values(["ANIO_EGRESO", "promedio_peso_grd"], ascending=False)
)

In [19]:
resumen_complejidad = df.merge(
    esquema_hospitales, how="left", left_on="COD_HOSPITAL", right_on="Código nuevo Establecimiento"
)

In [20]:
resumen_complejidad.to_excel("../data/interim/resumen_complejidad_pais.xlsx", index=False)